# Codes' functionality:
## Single out websites that 
* can not be opened(return 404) 
* or throw exception when requests.get(url)
#### Python get:
* try...except...finally syntax
* construct DataFrame

### Nice coding habits:
* truncate a small amount of data to process during testing stage,
** such as: urls = urls.loc[10:20] allows only 10 entries to be selected.

In [23]:
import pandas as pd
import webbrowser
from datetime import date
import os
import requests

In [3]:
def generate_excel(df, f_name):
    if (df.shape[0] == 0):
        print ('没有数据产出。')
    else:
        if os.path.exists(f_name):
            os.remove(f_name)
        df.to_excel(f_name, index=False)
        print(f_name, '文件已生成。')
#获得商户详情，如AM/BD
def get_mer_info():
    mer_info = pd.read_excel('资料汇总.xlsx', sheet_name='AM分配')
    mer_info = mer_info.applymap(str)
    mer_info['商户号'] = mer_info['商户号'].map(lambda x: x.replace('.0', ''))
    return mer_info
#获得已经关停的网站
def get_down_urls():
    down_urls = pd.read_excel('资料汇总.xlsx', sheet_name='关停网站')
    down_urls = down_urls.applymap(str)
    down_urls['商户号'] = down_urls['商户号'].map(lambda x: x.replace('.0', ''))
    return down_urls

In [25]:
#收集打不开的URL
urls = pd.read_excel('top4-11.xlsx')
urls = urls.applymap(str)
print('初始网址数量：', urls.shape[0])
urls.drop_duplicates('TRADEURL', inplace=True)
print('删除重复网址：', urls.shape[0])
urls.drop(urls[urls['TRADEURL'] == 'Total'].index, inplace=True)
down_urls = get_down_urls()
urls.drop(urls[urls['TRADEURL'].isin(down_urls['URL'])].index, inplace=True)
print('删除关停网址：', urls.shape[0])

urls.TRADEURL = urls.TRADEURL.map(lambda x: x.replace('www.', ''))
# urls.head(2)

# truncate a small amount of data to process during testing stage
# urls = urls.loc[10:20]
#判断打不开的
site_size = len(urls['TRADEURL'].unique())
print ('待查看的URL数量', site_size)
check_list = 0
url_404 = []
for i, link in enumerate(list(urls.TRADEURL.unique())):
#     print(link)
    mer_no = urls[urls.TRADEURL == link]['MER_NO'].unique()[0]
    try:
        resp = requests.get('https://www.' + link)
        if (resp.status_code == 404):
    # #         webbrowser.open('http://www.' + link)     
            record = pd.DataFrame({'商户号': [mer_no], 'URL': [link], '404': '打不开'})
            url_404.append(record)
            check_list = check_list + 1  
    except:
        record = pd.DataFrame({'商户号': [mer_no], 'URL': [link], '404': '出错了'})
        url_404.append(record)
        check_list = check_list + 1
#         webbrowser.open('http://www.' + link)
            
df_404 = pd.concat(url_404, axis=0, ignore_index=True )
print('待查看的404网站数量：', check_list)
print('过关的网站数量：', site_size - check_list)
mers = get_mer_info()
df_404 = df_404.applymap(str)
df = df_404.merge(mers[['AM', '商户号']], on='商户号', how='left')
f_name = str(date.today()) + '待查网站.xlsx'
generate_excel(df, f_name)

初始网址数量： 357
删除重复网址： 351
删除关停网址： 333
待查看的URL数量 322
待查看的404网站数量： 118
过关的网站数量： 204
2022-02-18待查网站.xlsx 文件已生成。
